In [31]:
import json
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [32]:
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL="llama3.2"
HEADERS={"Content-Type":"application/json"}

In [33]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:
    def __init__(self, url):
        self.url = url
        res = requests.get(self.url, headers=headers)
        self.body = res.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irr in soup.body(["script","style","input", "img"]):
            irr.decompose()
        self.text = soup.get_text(separator="\n", strip=True)
        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links]
    def get_contents(self):
        return f"The title of the webpage is {self.title} and the contents of the content of the webpage is {self.text}"

In [34]:
system_prompt_for_links = "You are an expert analyzer in selecting the links which are provided to you as a list, the links would be used for \
creating a company's brochure donot include conatct, privacy, testimonials or settings related links\
you should include links such as about page, career's page, comapny's page \
repond in STRICTLY JSON format and only give the useful links which are complete donot add any other texts or broken links with it"
system_prompt_for_links += "You are provided with an example respond in this particular format donot give extra inverted commas in your response which make it difficult to parse a JSON string "
system_prompt_for_links += """{
"links":[
{"type":"about page","url":"https://full url of about page/about"},
{"type":"career page","url":"https://full url of about page/career"},
]
}"""

In [35]:
def user_prompt_for_link(website):
    user_prompt = f"you are viewing a company's page whose title is {website.title}"
    user_prompt += " you have to pick the links which would be useful for building a company's brochure \
    repond STRICTLY in JSON format and donot give any other texts with you response just select the links and give the links and most important give the full link \
    to that page Donot give broken links \n"
    user_prompt += f"links are as follows {website.links}"
    return user_prompt

In [36]:
def link_messages(website):
    return [
        {"role":"system", "content":system_prompt_for_links},
        {"role":"user","content": user_prompt_for_link(website)}
    ]

In [37]:
def get_links(url):
    website = Website(url)
    res = requests.post(OLLAMA_API, json={"model":MODEL, "messages":link_messages(website), "stream":False}, headers=HEADERS)
    result = res.json()["message"]["content"]
    # print(result)
    raw_links = json.dumps(result)
    links = json.loads(raw_links)
    return links

In [38]:
def get_all_details(url):
    result = Website(url).get_contents()
    rawlinks = get_links(url)
    links = json.loads(rawlinks)
    for link in links["links"]:
        # print(link)
        result += f"you are viewing the page {link['type']}"
        result += Website(link["url"]).get_contents()
    return result

In [39]:
system_prompt_brochure = "you are an expert in analyzing the entire website content and generating the company's brochure \
the brochure should include what is company is about what is its culture career/jobs what it does and all the relevent information which you think \
is important for making a perfect brochure, Respond in MARKDOWN"

In [43]:
def user_prompt_for_brochure(company_name, url):
    user_prompt = f"You are making the brochure for this comapny {company_name}"
    user_prompt += " here is all the relevant information about the company from its different pages such as landing page, about page which you will require \
    to build the company's brochure, Respond in Markdown"
    user_prompt += get_all_details(url)
    return user_prompt
    

In [44]:
def messages_for_brochure(comapny_name, url):
    return [
        {"role":"system", "content": system_prompt_brochure},
        {"role":"user", "content":user_prompt_for_brochure(comapny_name, url)}
    ]

In [45]:
def company_brochure(comapny_name, url):
    res = requests.post(OLLAMA_API, json={"model": MODEL, "messages": messages_for_brochure(comapny_name, url), "stream":False}, headers=HEADERS)
    display(Markdown(res.json()["message"]["content"]))
company_brochure("Masai School","https://www.masaischool.com/")

It appears that you are looking for information about Masai School, a coding bootcamp that offers various courses in software development, data analytics, business analytics, digital marketing, and more.

Here are some answers to the questions you raised:

1. What is the medium of instruction?

Masai School offers classes in English.

2. Do you offer loaner laptops for prospective students?

Yes, Masai School provides laptop loans to students who need them.

3. Does college CGPA play any role in the program?

No, college GPA does not impact admission to Masai School's programs.

4. Will I receive a degree or certificate when I graduate from Masai?

Masai School offers certificates upon graduation for most programs.

5. What do I do if I don't have a Pan Card/Adhaar Card?

Students can apply for a PAN card and Adhar card at the time of admission to verify their identity.

6. What do you look for in candidates when they apply to Masai?

Masai looks for candidates with basic knowledge of programming concepts, good communication skills, a willingness to learn, and strong motivation.

7. What is the course fee?

The cost of courses varies depending on the program, but here are some approximate fees:

* Software Development Program: ₹4 lakhs - ₹5 lakhs
* Data Analytics Program: ₹3 lakhs - ₹4 lakhs
* Business Analytics Program: ₹2 lakhs - ₹3 lakhs

8. The course hours don't work for me. Can I attend classes in my own time?

Masai offers flexible scheduling options, including part-time and full-time programs.

9. Where is Masai located?

Masai School has campuses in Bengaluru, India, and online classes are also available.

10. Is attendance mandatory?

Yes, regular attendance is expected at Masai School.

11. What is the duration of the program?

The duration of courses varies depending on the program:

* Software Development Program: 12-18 months
* Data Analytics Program: 9-12 months
* Business Analytics Program: 6-9 months

12. What if I leave the course in between?

Students can withdraw from the program at any time, but they will need to pay a penalty.

13. Do I have to take the first job I am offered?

No, Masai does not guarantee placement for graduates. However, they do offer career support and resources to help students find employment.

14. Does Masai help me get a job?

Yes, Masai School provides career support and resources, including resume building, interview practice, and job placement assistance.

15. What computer specs do I need?

For the Software Development Program, students will need:

* A laptop with Intel Core i5 or i7 processor
* 8 GB RAM or more
* 256 GB SSD storage

For other programs, specific hardware requirements may vary.

16. Who is eligible to apply for a Masai course?

Anyone with basic knowledge of programming concepts and a willingness to learn can apply for Masai School's courses.

17. What is the difference between Masai One and Masai Launchpad?

Masai One is an online program that provides flexible learning options, while Masai Launchpad is a full-time program offered in Bengaluru.